<a href="https://colab.research.google.com/github/rohan-kalra904/RAG-Pipeline/blob/main/RAG_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community

!pip install pypdf
!pip install chromadb

!pip install --upgrade sentence-transformers InstructorEmbedding

  Using cached InstructorEmbedding-1.0.1-py2.py3-none-any.whl.metadata (20 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-

In [2]:
!pip install "pinecone[grpc]"
!pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.4/421.4 kB 24.0 MB/s eta 0:00:00


In [3]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00


In [4]:
'''Setting up Pinecone Database'''
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time

# Initialize a pinecode instance and it's index
pc = Pinecone(api_key="pinecone-api-key")
index_name = "medical-rag"

# Here we create a serverless index
if pc.has_index(index_name):
  pc.delete_index(index_name)
pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)


# Wait for index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

# Target the index
index = pc.Index(index_name)

In [6]:
# Initialize MedEmbed model and tokenizer
from bs4 import BeautifulSoup
import PyPDF2
import requests
import io
from transformers import AutoTokenizer, AutoModel
import torch
def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()

    # Potentially more text formatting functions can go here
    return cleaned_text

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [13]:
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from PyPDF2 import PdfReader
import uuid

def extract_text_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.get_text()

def extract_text_from_pdf(pdf_file):
    reader = PyPDF2.PdfReader(io.BytesIO(pdf_file.read()))
    return " ".join([page.extract_text() for page in reader.pages])

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip()

    # Potentially more text formatting functions can go here
    return cleaned_text


data = []

def chunk_document(text, metadata, max_chunk_size=400):
    # Tokenize the entire text
    tokens = tokenizer.tokenize(text)
    chunks = []

    for i in range(0, len(tokens), max_chunk_size):
        chunk_tokens = tokens[i:i + max_chunk_size]
        chunk_text = tokenizer.convert_tokens_to_string(chunk_tokens)

        # Create chunk-specific metadata
        chunk_metadata = metadata.copy()  # Preserve original metadata
        chunk_metadata.update({
            "chunk_id": str(uuid.uuid4()),
            "chunk_index": i // max_chunk_size,
            "total_chunks": (len(tokens) - 1) // max_chunk_size + 1
        })

        chunks.append({
            "text": chunk_text,
            "metadata": chunk_metadata
        })

    return chunks

#Function to add a new document in the existing database of documents
def add_document(source,content_type):
    if content_type == 'pdf':
        with open(source, 'rb') as file:
            text = text_formatter(extract_text_from_pdf(file))
            reader = PdfReader(source)
            metadata = reader.metadata
            # title = metadata.title
            # author = metadata.author
            # data.append({"title":title,"author":author,"text":text})
            metadata = {"title":"" if metadata.title==None else metadata.title,"author":"" if metadata.author==None else metadata.author}
            chunks = chunk_document(text, metadata)
            data.extend(chunks)

    print(print(f"Added document from {source}"))



In [14]:
#Add all documents here
add_document('/content/drive/MyDrive/main.pdf','pdf')
add_document('/content/drive/MyDrive/song-et-al-2017-progress-in-the-treatment-of-advanced-gastric-cancer.pdf','pdf')
add_document('/content/drive/MyDrive/duffy-et-al-2023-circulating-tumor-dna-(ctdna)-as-a-biomarker-for-lung-cancer-early-detection-monitoring-and-therapy.pdf','pdf')
add_document('/content/drive/MyDrive/9789241550390-eng.pdf','pdf')
add_document('/content/drive/MyDrive/tripodi-et-al-2022-the-role-of-nutrition-in-children-with-cancer.pdf','pdf')
add_document('/content/drive/MyDrive/9789240030824-eng.pdf','pdf')
add_document('/content/drive/MyDrive/ESPEN-practical-guideline-clinical-nutrition-in-cancer.pdf','pdf')
add_document('/content/drive/MyDrive/Breast Cancer Treatment - NCI.pdf','pdf')
add_document('/content/drive/MyDrive/recist_guideline.pdf','pdf')
add_document('/content/drive/MyDrive/Book4.pdf','pdf')
add_document('/content/drive/MyDrive/Book2.pdf','pdf')
add_document('/content/drive/MyDrive/Testicular Cancer Treatment - NCI.pdf','pdf')
add_document('/content/drive/MyDrive/Treatment of Stomach Cancer - NCI.pdf','pdf')
add_document('/content/drive/MyDrive/Esophageal Cancer Treatment - NCI.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')
# add_document('/content/Medical Pdfs/main.pdf','pdf')




Added document from /content/drive/MyDrive/main.pdf
None
Added document from /content/drive/MyDrive/song-et-al-2017-progress-in-the-treatment-of-advanced-gastric-cancer.pdf
None
Added document from /content/drive/MyDrive/duffy-et-al-2023-circulating-tumor-dna-(ctdna)-as-a-biomarker-for-lung-cancer-early-detection-monitoring-and-therapy.pdf
None
Added document from /content/drive/MyDrive/9789241550390-eng.pdf
None
Added document from /content/drive/MyDrive/tripodi-et-al-2022-the-role-of-nutrition-in-children-with-cancer.pdf
None
Added document from /content/drive/MyDrive/9789240030824-eng.pdf
None
Added document from /content/drive/MyDrive/ESPEN-practical-guideline-clinical-nutrition-in-cancer.pdf
None
Added document from /content/drive/MyDrive/Breast Cancer Treatment - NCI.pdf
None
Added document from /content/drive/MyDrive/recist_guideline.pdf
None
Added document from /content/drive/MyDrive/Book4.pdf
None
Added document from /content/drive/MyDrive/Book2.pdf
None
Added document from /c

In [15]:

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = embedding_model(**inputs)
        # Check the correct attribute for embeddings in MedEmbed's output
        embedding = outputs.pooler_output.squeeze().tolist()
    return embedding



all_embeddings = []
for chunk in data:
    embedding = generate_embedding(chunk['text'])
    all_embeddings.append({"id":str(len(all_embeddings)),"values":embedding,"metadata":{**chunk["metadata"],"source_text":chunk['text']}})

In [16]:
for embedding in all_embeddings:
  print(embedding['metadata'])

{'title': 'Bilateral Adrenal Tumors: A Visual Case Series', 'author': 'Dipika R. Mohan MD PhD', 'chunk_id': '51cd555b-feed-4ad1-8d0b-031d5105db9b', 'chunk_index': 0, 'total_chunks': 20, 'source_text': 'visual vignette bilateral adrenal tumors : a visual case series dipika r. mohan, md, phd1, rutu shah, md2, malak itani, md3, mohamed awali, md3, sina jasim, md, mph2, * 1department of medicine, school of medicine, washington university in st. louis, st. louis, missouri 2division of endocrinology, metabolism and lipid research, school of medicine, washington university in st. louis, st. louis, missouri 3mallinckrodt institute of radiology, washington university in st. louis, st. louis, missouri article info article history : received 13 november 2024received in revised form 18 november 2024 accepted 19 november 2024available online 27 november 2024 key words : adrenal adenomapheochromocytoma imaging introduction incidental adrenal masses are common, affecting > 6 % of those above 60 years

In [17]:
import random
import itertools

#Upsert the generated vectors into the pinecone database
#Improvement - Upsert the batches of vectors in parallel for improved speed
def chunks(iterable, batch_size=200):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))


batch_size = 200
for chunk in chunks(all_embeddings, batch_size):
    index.upsert(vectors=chunk,namespace = 'medical-references')

time.sleep(10)  # Wait for the upserted vectors to be indexed


In [18]:
print(index.describe_index_stats())

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'medical-references': {'vector_count': 0}},
 'total_vector_count': 0}


In [19]:
def retrieve_result(query_text,top_k=3):

  # Generate embedding for query
  query_embedding = generate_embedding(query_text)

  # Query Pinecone
  results = index.query(namespace = "medical-references",vector=query_embedding, top_k=top_k,include_values=False, include_metadata=True)
  # print(results)
  return results


In [20]:
# Set up openai api
# Research on open source LLMs more suitable for the task
import os
import openai
os.environ["OPENAI_API_KEY"] = "openai-api-key"

client = openai.Client()

def generate_response(prompt, model="gpt-4o-mini"):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful medical assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=500,
        temperature=0.7
    )
    print(response)
    return response.choices[0].message.content.strip()


In [21]:


def rag_pipeline(query, retrieved_chunks):
    # Extract source text from metadata of each match
    retrieved_texts = [match['metadata']['source_text'] for match in retrieved_chunks['matches']]
    retrieved_titles =  [match['metadata']['title'] for match in retrieved_chunks['matches']]
    context = "\n".join(retrieved_texts)  # Join the extracted texts
    references = "\n".join(context)
    prompt = f"""
    Context: {context}
    References: {references}
    Question: {query}

    Instructions:
    1. Answer the question using only the provided context and generally accepted medical knowledge.
    2. If uncertain, state that you don't have enough information to answer accurately.
    3. Include references to the original sources in your answer.
    4. Suggest 2-3 relevant follow-up questions.
    5. Add a disclaimer that this is not official medical advice.

    Answer:
    """
    response = generate_response(prompt)
    return response


In [22]:
  def run_instance(user_query):
      relevant_chunks = retrieve_result(user_query)  # Implement this function
      answer = rag_pipeline(user_query, relevant_chunks)
      return answer


In [23]:
# Tester Code
user_query = input("Enter the query")
print(run_instance(user_query))

Enter the queryWhat are the early warning signs of lung cancer?


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}